In [1]:
from sklearn.cluster import KMeans

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from gmplot import GoogleMapPlotter

%load_ext autoreload
%autoreload 2

import json

In [3]:
bnb_fl = '../data/bnb-seattle.json'
seattle_center = '47.629458, -122.336338'

In [4]:
with open(bnb_fl) as busfile:
    bnb_json = json.load(busfile)
    bnb_json_results = bnb_json['search_results']
    bnb_seattle = pd.io.json.json_normalize(bnb_json_results)


In [16]:
bnb_seattle_ = bnb_seattle[bnb_seattle['listing.city'].str.lower() == 'seattle']
bnb_seattle_.shape

(2028, 64)

In [20]:
bnb_seattle_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2028 entries, 0 to 2370
Data columns (total 64 columns):
listing.bathrooms                                   2028 non-null float64
listing.bedrooms                                    2028 non-null int64
listing.beds                                        2028 non-null float64
listing.city                                        2028 non-null object
listing.distance                                    0 non-null object
listing.extra_host_languages                        2028 non-null object
listing.id                                          2028 non-null int64
listing.instant_bookable                            2028 non-null bool
listing.is_business_travel_ready                    2028 non-null bool
listing.is_new_listing                              2028 non-null bool
listing.lat                                         2028 non-null float64
listing.listing_tags                                2028 non-null object
listing.lng              

In [21]:
keep_cols = ['listing.bathrooms', 'listing.beds', 'listing.lat', 'listing.lng', 'listing.reviews_count'
             ,'listing.room_type', 'listing.room_type_category', 'pricing_quote.total_price']

bnb_seattle_reduced = bnb_seattle_[keep_cols]

In [23]:
bnb_seattle_reduced['listing.room_type_category'].value_counts()

entire_home     1588
private_room     425
shared_room       15
Name: listing.room_type_category, dtype: int64

In [13]:
bnb_panda['pricing_quote.guests'].value_counts()

1    2372
Name: pricing_quote.guests, dtype: int64

In [4]:
import pyspark.sql

In [5]:
bnb_panda['listing.public_address']

bnb_list_seattle = bnb_panda[bnb_panda['listing.public_address'].str.contains('Seattle')]

In [82]:
bnb_list_seattle['listing.lng'].min()

-122.4209176433197

In [128]:
map_location = bnb_list_seattle[['listing.lng', 'listing.lat']]
map_location = map_location[map_location['listing.lat'] > 47.515]
map_location = map_location[map_location['listing.lat'] < 47.69468]
map_location = map_location[map_location['listing.lat'] > -122.420]
map_location = map_location[map_location['listing.lat'] < -122.222]



bnb_cluster = KMeans(n_clusters=9)

bnb_cluster.fit(map_location)

what_is_this = bnb_cluster.fit_predict(bnb_list_seattle[['listing.lng', 'listing.lat']])
clustered_lats = []
clustered_longs = []
for n, i in enumerate(what_is_this):
    lats = bnb_list_seattle[what_is_this == n]['listing.lat'].values
    longs = bnb_list_seattle[what_is_this == n]['listing.lng'].values
    clustered_lats.append(lats)
    clustered_longs.append(longs)

colors_list = ['crimson', 'darkseagreen', 
               'firebrick', 'dodgerblue', 
               'burlywood' 'gold',
               'indigo',  'navy', 
               'indianred', 'lawngreen']

longs = bnb_list_seattle['listing.lng'].values
lats = bnb_list_seattle['listing.lat'].values

s_lat = '47.629458' 
s_long = '-122.336338'

cent_long = bnb_cluster.cluster_centers_[:,0]
cent_lat = bnb_cluster.cluster_centers_[:,1]

gmap = GoogleMapPlotter(s_lat, s_long, 11.8)

for color, lats, longs in zip(colors_list, clustered_lats, clustered_longs):
    gmap.scatter(lats, longs, color, size=5, marker=False)

# gmap.plot(lats, longs, 'cornflowerblue', edge_width=10)
# gmap.scatter(lats, longs, '#3B0B39', size=40, marker=False)
# gmap.scatter(lats, longs, 'k', marker=True)
# gmap.heatmap(lats, longs)
c_longs = []
c_lats = []
for line in bnb_cluster.cluster_centers_:
    c_longs.append(line[0])
    c_lats.append(line[1])
gmap.scatter(c_lats, c_longs, 'hotpink', size=40, marker=True)

gmap.draw("../plots/mymap_bnb_centers.html")

ValueError: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.